In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import skew
from scipy.stats import norm
from scipy.stats.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
%matplotlib inline
import sys
sys.path.append("/content/drive/MyDrive/MIDA2/IntelligentMobilityProject/Code")
np.set_printoptions(precision=5, suppress=True)

In [ ]:
algo = "HC"
sep = "_"
metric = "HEOM"
vars = 6
regions = True
if vars != 7:
  regions = False
if_regions = "_regions" if regions else ""

file_type = ".npy"
linkages = ["single", "complete", "average", "weighted"]#, "ward", "centroid", "median"]

base_path = "/content/drive/MyDrive/MIDA2/IntelligentMobilityProject/"
pictures_path = f"{base_path}Pictures/"
dataset_path = f"{base_path}Survey+dataset/"
results_path = f"{base_path}Code/Data/"
HC_base_path = f"{results_path}{algo}/"

In [ ]:
ds_path = f'{dataset_path}4_DNA_{vars}values{if_regions}_v2.csv'

df_DNA = pd.read_csv(ds_path)

In [ ]:
df_DNA

,InternetUsers,Concern_environmental_impacts,grouped_Age,Would_subsribe_car_sharing_if_available,Preference_tolls_or_traffic_limitation,Gender
0,0.758621,0.4,0.59,0.333333,0.00,0.0
1,0.689655,0.8,0.11,0.333333,0.25,1.0
2,0.551724,0.6,0.59,0.166667,0.00,1.0
3,0.931034,0.8,1.00,0.000000,0.50,1.0
4,0.327586,0.8,0.35,0.333333,0.00,1.0
...,...,...,...,...,...,...
26600,0.551724,0.2,0.18,0.666667,0.50,0.0
26601,0.551724,0.4,0.26,0.000000,0.00,0.0
26602,0.551724,0.7,0.26,0.333333,0.00,1.0
26603,0.551724,0.7,0.35,0.666667,0.25,1.0


# Clustering


## Hierarchical Clustering

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, inconsistent
from scipy.spatial.distance import cdist, pdist, squareform

 ### Dendogram and WSS/BSS

#### Plotly dendrogram

In [ ]:
# -*- coding: utf-8 -*-

from __future__ import absolute_import

from collections import OrderedDict

from plotly import exceptions, optional_imports
from plotly.graph_objs import graph_objs as go

# Optional imports, may be None for users that only use our core functionality.
np = optional_imports.get_module("numpy")
scp = optional_imports.get_module("scipy")
sch = optional_imports.get_module("scipy.cluster.hierarchy")
scs = optional_imports.get_module("scipy.spatial")


def create_dendrogram(
    X,
    orientation="bottom",
    labels=None,
    colorscale=None,
    distfun=None,
    linkagefun=lambda x: sch.linkage(x, "complete"),
    hovertext=None,
    color_threshold=None,
    p=None,
    truncate_mode=None
):
    """
    Function that returns a dendrogram Plotly figure object. This is a thin
    wrapper around scipy.cluster.hierarchy.dendrogram.

    See also https://dash.plot.ly/dash-bio/clustergram.

    :param (ndarray) X: Matrix of observations as array of arrays
    :param (str) orientation: 'top', 'right', 'bottom', or 'left'
    :param (list) labels: List of axis category labels(observation labels)
    :param (list) colorscale: Optional colorscale for the dendrogram tree.
                              Requires 8 colors to be specified, the 7th of
                              which is ignored.  With scipy>=1.5.0, the 2nd, 3rd
                              and 6th are used twice as often as the others.
                              Given a shorter list, the missing values are
                              replaced with defaults and with a longer list the
                              extra values are ignored.
    :param (function) distfun: Function to compute the pairwise distance from
                               the observations
    :param (function) linkagefun: Function to compute the linkage matrix from
                               the pairwise distances
    :param (list[list]) hovertext: List of hovertext for constituent traces of dendrogram
                               clusters
    :param (double) color_threshold: Value at which the separation of clusters will be made

    Example 1: Simple bottom oriented dendrogram

    >>> from plotly.figure_factory import create_dendrogram

    >>> import numpy as np

    >>> X = np.random.rand(10,10)
    >>> fig = create_dendrogram(X)
    >>> fig.show()

    Example 2: Dendrogram to put on the left of the heatmap
    
    >>> from plotly.figure_factory import create_dendrogram

    >>> import numpy as np

    >>> X = np.random.rand(5,5)
    >>> names = ['Jack', 'Oxana', 'John', 'Chelsea', 'Mark']
    >>> dendro = create_dendrogram(X, orientation='right', labels=names)
    >>> dendro.update_layout({'width':700, 'height':500}) # doctest: +SKIP
    >>> dendro.show()

    Example 3: Dendrogram with Pandas
    
    >>> from plotly.figure_factory import create_dendrogram

    >>> import numpy as np
    >>> import pandas as pd

    >>> Index= ['A','B','C','D','E','F','G','H','I','J']
    >>> df = pd.DataFrame(abs(np.random.randn(10, 10)), index=Index)
    >>> fig = create_dendrogram(df, labels=Index)
    >>> fig.show()
    """
    if not scp or not scs or not sch:
        raise ImportError(
            "FigureFactory.create_dendrogram requires scipy, \
                            scipy.spatial and scipy.hierarchy"
        )

    s = X.shape
    if len(s) != 2:
        exceptions.PlotlyError("X should be 2-dimensional array.")

    if distfun is None:
        distfun = scs.distance.pdist

    dendrogram = _Dendrogram(
        X,
        orientation,
        labels,
        colorscale,
        distfun=distfun,
        linkagefun=linkagefun,
        hovertext=hovertext,
        color_threshold=color_threshold,
        p=p,
        truncate_mode=truncate_mode
    )

    return go.Figure(data=dendrogram.data, layout=dendrogram.layout)


class _Dendrogram(object):
    """Refer to FigureFactory.create_dendrogram() for docstring."""

    def __init__(
        self,
        X,
        orientation="bottom",
        labels=None,
        colorscale=None,
        width=np.inf,
        height=np.inf,
        xaxis="xaxis",
        yaxis="yaxis",
        distfun=None,
        linkagefun=lambda x: sch.linkage(x, "complete"),
        hovertext=None,
        color_threshold=None,
        p=None,
        truncate_mode=None
    ):
        self.orientation = orientation
        self.labels = labels
        self.xaxis = xaxis
        self.yaxis = yaxis
        self.data = []
        self.leaves = []
        self.sign = {self.xaxis: 1, self.yaxis: 1}
        self.layout = {self.xaxis: {}, self.yaxis: {}}

        if self.orientation in ["left", "bottom"]:
            self.sign[self.xaxis] = 1
        else:
            self.sign[self.xaxis] = -1

        if self.orientation in ["right", "bottom"]:
            self.sign[self.yaxis] = 1
        else:
            self.sign[self.yaxis] = -1

        if distfun is None:
            distfun = scs.distance.pdist

        (dd_traces, xvals, yvals, ordered_labels, leaves) = self.get_dendrogram_traces(
            X, colorscale, distfun, linkagefun, hovertext, color_threshold, p, truncate_mode
        )

        self.labels = ordered_labels
        self.leaves = leaves
        yvals_flat = yvals.flatten()
        xvals_flat = xvals.flatten()

        self.zero_vals = []

        for i in range(len(yvals_flat)):
            if yvals_flat[i] == 0.0 and xvals_flat[i] not in self.zero_vals:
                self.zero_vals.append(xvals_flat[i])

        if len(self.zero_vals) > len(yvals) + 1:
            # If the length of zero_vals is larger than the length of yvals,
            # it means that there are wrong vals because of the identicial samples.
            # Three and more identicial samples will make the yvals of spliting
            # center into 0 and it will accidentally take it as leaves.
            l_border = int(min(self.zero_vals))
            r_border = int(max(self.zero_vals))
            correct_leaves_pos = range(
                l_border, r_border + 1, int((r_border - l_border) / len(yvals))
            )
            # Regenerating the leaves pos from the self.zero_vals with equally intervals.
            self.zero_vals = [v for v in correct_leaves_pos]

        self.zero_vals.sort()
        self.layout = self.set_figure_layout(width, height)
        self.data = dd_traces

    def get_color_dict(self, colorscale):
        """
        Returns colorscale used for dendrogram tree clusters.

        :param (list) colorscale: Colors to use for the plot in rgb format.
        :rtype (dict): A dict of default colors mapped to the user colorscale.

        """

        # These are the color codes returned for dendrograms
        # We're replacing them with nicer colors
        # This list is the colors that can be used by dendrogram, which were
        # determined as the combination of the default above_threshold_color and
        # the default color palette (see scipy/cluster/hierarchy.py)
        d = {
            "r": "red",
            "g": "green",
            "b": "blue",
            "c": "cyan",
            "m": "magenta",
            "y": "yellow",
            "k": "black",
            # TODO: 'w' doesn't seem to be in the default color
            # palette in scipy/cluster/hierarchy.py
            "w": "white",
        }
        default_colors = OrderedDict(sorted(d.items(), key=lambda t: t[0]))

        if colorscale is None:
            rgb_colorscale = [
                "rgb(0,116,217)",  # blue
                "rgb(35,205,205)",  # cyan
                "rgb(61,153,112)",  # green
                "rgb(40,35,35)",  # black
                "rgb(133,20,75)",  # magenta
                "rgb(255,65,54)",  # red
                "rgb(255,255,255)",  # white
                "rgb(255,220,0)",  # yellow
            ]
        else:
            rgb_colorscale = colorscale

        for i in range(len(default_colors.keys())):
            k = list(default_colors.keys())[i]  # PY3 won't index keys
            if i < len(rgb_colorscale):
                default_colors[k] = rgb_colorscale[i]

        # add support for cyclic format colors as introduced in scipy===1.5.0
        # before this, the colors were named 'r', 'b', 'y' etc., now they are
        # named 'C0', 'C1', etc. To keep the colors consistent regardless of the
        # scipy version, we try as much as possible to map the new colors to the
        # old colors
        # this mapping was found by inpecting scipy/cluster/hierarchy.py (see
        # comment above).
        new_old_color_map = [
            ("C0", "b"),
            ("C1", "g"),
            ("C2", "r"),
            ("C3", "c"),
            ("C4", "m"),
            ("C5", "y"),
            ("C6", "k"),
            ("C7", "g"),
            ("C8", "r"),
            ("C9", "c"),
        ]
        for nc, oc in new_old_color_map:
            try:
                default_colors[nc] = default_colors[oc]
            except KeyError:
                # it could happen that the old color isn't found (if a custom
                # colorscale was specified), in this case we set it to an
                # arbitrary default.
                default_colors[n] = "rgb(0,116,217)"

        return default_colors

    def set_axis_layout(self, axis_key):
        """
        Sets and returns default axis object for dendrogram figure.

        :param (str) axis_key: E.g., 'xaxis', 'xaxis1', 'yaxis', yaxis1', etc.
        :rtype (dict): An axis_key dictionary with set parameters.

        """
        axis_defaults = {
            "type": "linear",
            "ticks": "outside",
            "mirror": "allticks",
            "rangemode": "tozero",
            "showticklabels": True,
            "zeroline": False,
            "showgrid": False,
            "showline": True,
        }

        if len(self.labels) != 0:
            axis_key_labels = self.xaxis
            if self.orientation in ["left", "right"]:
                axis_key_labels = self.yaxis
            if axis_key_labels not in self.layout:
                self.layout[axis_key_labels] = {}
            self.layout[axis_key_labels]["tickvals"] = [
                zv * self.sign[axis_key] for zv in self.zero_vals
            ]
            self.layout[axis_key_labels]["ticktext"] = self.labels
            self.layout[axis_key_labels]["tickmode"] = "array"

        self.layout[axis_key].update(axis_defaults)

        return self.layout[axis_key]

    def set_figure_layout(self, width, height):
        """
        Sets and returns default layout object for dendrogram figure.

        """
        self.layout.update(
            {
                "showlegend": False,
                "autosize": False,
                "hovermode": "closest",
                "width": width,
                "height": height,
            }
        )

        self.set_axis_layout(self.xaxis)
        self.set_axis_layout(self.yaxis)

        return self.layout

    def get_dendrogram_traces(
        self, X, colorscale, distfun, linkagefun, hovertext, color_threshold, p, truncate_mode
    ):
        """
        Calculates all the elements needed for plotting a dendrogram.

        :param (ndarray) X: Matrix of observations as array of arrays
        :param (list) colorscale: Color scale for dendrogram tree clusters
        :param (function) distfun: Function to compute the pairwise distance
                                   from the observations
        :param (function) linkagefun: Function to compute the linkage matrix
                                      from the pairwise distances
        :param (list) hovertext: List of hovertext for constituent traces of dendrogram
        :rtype (tuple): Contains all the traces in the following order:
            (a) trace_list: List of Plotly trace objects for dendrogram tree
            (b) icoord: All X points of the dendrogram tree as array of arrays
                with length 4
            (c) dcoord: All Y points of the dendrogram tree as array of arrays
                with length 4
            (d) ordered_labels: leaf labels in the order they are going to
                appear on the plot
            (e) P['leaves']: left-to-right traversal of the leaves

        """
        d = distfun(X)
        Z = linkagefun(d)
        P = sch.dendrogram(
            Z,
            orientation=self.orientation,
            labels=self.labels,
            no_plot=True,
            color_threshold=color_threshold,
            p=p,
            truncate_mode=truncate_mode
        )

        icoord = scp.array(P["icoord"])
        dcoord = scp.array(P["dcoord"])
        ordered_labels = scp.array(P["ivl"])
        color_list = scp.array(P["color_list"])
        colors = self.get_color_dict(colorscale)

        trace_list = []

        for i in range(len(icoord)):
            # xs and ys are arrays of 4 points that make up the '∩' shapes
            # of the dendrogram tree
            if self.orientation in ["top", "bottom"]:
                xs = icoord[i]
            else:
                xs = dcoord[i]

            if self.orientation in ["top", "bottom"]:
                ys = dcoord[i]
            else:
                ys = icoord[i]
            color_key = color_list[i]
            hovertext_label = None
            if hovertext:
                hovertext_label = hovertext[i]
            trace = dict(
                type="scatter",
                x=np.multiply(self.sign[self.xaxis], xs),
                y=np.multiply(self.sign[self.yaxis], ys),
                mode="lines",
                marker=dict(color=colors[color_key]),
                text=hovertext_label,
                hoverinfo="text",
            )

            try:
                x_index = int(self.xaxis[-1])
            except ValueError:
                x_index = ""

            try:
                y_index = int(self.yaxis[-1])
            except ValueError:
                y_index = ""

            trace["xaxis"] = "x" + x_index
            trace["yaxis"] = "y" + y_index

            trace_list.append(trace)

        return trace_list, icoord, dcoord, ordered_labels, P["leaves"]

#### Plot dendrogram

In [ ]:
def PlotDendrogram(Z, title="", save_path=None):
    plt.figure(figsize=(25, 10))
    font = {'family' : 'sans', 'size'   : 18}
    plt.rc('font', **font)
    plt.title('Hierarchical Clustering Dendrogram')
    plt.xlabel('Sample Index')
    plt.ylabel('Merge Distance')
    dendrogram(Z, 
        leaf_rotation=90.,  # rotates the x axis labels
        leaf_font_size=8.,  # font size for the x axis labels
    )
    plt.xticks([])
    plt.show()

    plt.savefig(save_path)

In [ ]:
def plotDendrogramPlotly(X, distance_matrix=None, labels=None, p=30, truncate_mode=None, color_threshold=None, show_heatmap=False, show_fig=True, save_path=None):
  #if labels == None:
  #  labels = [""]*X.shape[0]
  
  # Initialize figure by creating upper dendrogram
  fig = create_dendrogram(X, labels=labels, orientation='bottom', color_threshold=color_threshold, p=p,
                          truncate_mode=truncate_mode)
  if show_heatmap:
    for i in range(len(fig['data'])):
        fig['data'][i]['yaxis'] = 'y2'

    # Create Side Dendrogram
    dendro_side = create_dendrogram(X, labels=labels, orientation='right', color_threshold=color_threshold, p=p,
                            truncate_mode=truncate_mode)
    for i in range(len(dendro_side['data'])):
        dendro_side['data'][i]['xaxis'] = 'x2'

    # Add Side Dendrogram Data to Figure
    for data in dendro_side['data']:
        fig.add_trace(data)

    # Create Heatmap
    dendro_leaves = dendro_side['layout']['yaxis']['ticktext']
    print(dendro_leaves)
    dendro_leaves = list(map(int, dendro_leaves))
    heat_data = distance_matrix
    heat_data = heat_data[dendro_leaves,:]
    heat_data = heat_data[:,dendro_leaves]

    heatmap = [
        go.Heatmap(
            x = dendro_leaves,
            y = dendro_leaves,
            z = heat_data,
            colorscale = 'Blues'
        )
    ]

    heatmap[0]['x'] = fig['layout']['xaxis']['tickvals']
    heatmap[0]['y'] = dendro_side['layout']['yaxis']['tickvals']

    # Add Heatmap Data to Figure
    for data in heatmap:
        fig.add_trace(data)

    # Edit Layout
    fig.update_layout({'width':800, 'height':800,
                            'showlegend':False, 'hovermode': 'closest',
                            })
    # Edit xaxis
    fig.update_layout(xaxis={'domain': [.15, 1],
                                      'mirror': False,
                                      'showgrid': False,
                                      'showline': False,
                                      'zeroline': False,
                                      'ticks':""})
    # Edit xaxis2
    fig.update_layout(xaxis2={'domain': [0, .15],
                                      'mirror': False,
                                      'showgrid': False,
                                      'showline': False,
                                      'zeroline': False,
                                      'showticklabels': False,
                                      'ticks':""})

    # Edit yaxis
    fig.update_layout(yaxis={'domain': [0, .85],
                                      'mirror': False,
                                      'showgrid': False,
                                      'showline': False,
                                      'zeroline': False,
                                      'showticklabels': False,
                                      'ticks': ""
                            })
    # Edit yaxis2
    fig.update_layout(yaxis2={'domain':[.825, .975],
                                      'mirror': False,
                                      'showgrid': False,
                                      'showline': False,
                                      'zeroline': False,
                                      'showticklabels': False,
                                      'ticks':""})


  fig.update_layout({'width':1280, 'height':720}) 

  if show_fig:
    fig.show()

  if save_path:
    fig.write_html(save_path)

In [ ]:
X = np.random.rand(10,10)

In [ ]:
dM = squareform(pdist(X))
dM

array([[0.     , 1.56461, 1.3002 , 1.16152, 1.18636, 1.50616, 1.47003,
        1.2398 , 1.49225, 1.68163],
       [1.56461, 0.     , 1.03452, 1.58487, 1.46206, 1.44219, 1.25393,
        1.26257, 1.43863, 0.92211],
       [1.3002 , 1.03452, 0.     , 1.13205, 0.9865 , 1.32501, 1.1134 ,
        0.6769 , 1.01742, 1.00556],
       [1.16152, 1.58487, 1.13205, 0.     , 1.25573, 1.069  , 1.53322,
        1.29832, 1.34998, 1.38455],
       [1.18636, 1.46206, 0.9865 , 1.25573, 0.     , 1.43667, 1.31016,
        1.33456, 1.03749, 1.61817],
       [1.50616, 1.44219, 1.32501, 1.069  , 1.43667, 0.     , 1.96439,
        1.69849, 1.25759, 1.20969],
       [1.47003, 1.25393, 1.1134 , 1.53322, 1.31016, 1.96439, 0.     ,
        1.27417, 1.55038, 1.47226],
       [1.2398 , 1.26257, 0.6769 , 1.29832, 1.33456, 1.69849, 1.27417,
        0.     , 1.45336, 1.32865],
       [1.49225, 1.43863, 1.01742, 1.34998, 1.03749, 1.25759, 1.55038,
        1.45336, 0.     , 1.5788 ],
       [1.68163, 0.92211, 1.00556, 1.

In [ ]:
plotDendrogramPlotly(X, dM)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:358: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:359: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:360: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:361: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead



In [ ]:
plotDendrogramPlotly(X, dM, p=2,truncate_mode='level')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:358: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:359: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:360: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:361: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead



In [ ]:
plotDendrogramPlotly(X, dM, p=2,truncate_mode='level', show_heatmap=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:358: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:359: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:360: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:361: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead



['3' '(2)' '(3)' '7' '9' '(2)']


ValueError: ignored

In [ ]:
import time

def ComputeWSSBSS(X, pdist_matrix_sqr, Z,metric, k_values=range(1,20)):
    wss_values = []
    bss_values = []

    for k in k_values:
        clustering = fcluster(Z, k, criterion='maxclust')
        frequency = np.bincount(clustering)
        index = np.nonzero(frequency)[0]
        print("frequency: ", frequency)
        print("index: ", index)

        # USE MEDOIDS
        # numpy.argmin(distMatrix.sum(axis=0))
        centroids = [np.mean(X[clustering==c],axis=0) for c in index]
        print("centroids: ", centroids)
        #cdist(X, centroids, 'euclidean')
        D = cdist(X, centroids, 'euclidean')
        print("D: ", D)
        cIdx = np.argmin(D,axis=1)
        print("cIdx: ", cIdx)
        d = np.min(D,axis=1)
        print("d: ", d)

        avgWithinSS = sum(d)/len(X)
        print("avgWithinSS: ", avgWithinSS)
        # Total with-in sum of square
        wss = sum(d**2)
        print("wss: ", wss)
        tss = np.sum(pdist_matrix_sqr)/len(X)
        print("tss: ", tss)

        bss = tss-wss
        print("bss: ", bss)
        wss_values += [wss]
        bss_values += [bss]
    return wss_values,bss_values

In [ ]:
def PlotKneeElbow(bss_values,wss_values,k_values,title="", save_path=None):
    fig = plt.figure(figsize=(24,12))
    font = {'family' : 'sans', 'size'   : 12}
    plt.rc('font', **font)
    plt.plot(k_values, wss_values, 'bo-', color='red', label='WSS')
    plt.plot(k_values, bss_values, 'bo-', color='blue', label='BSS')
    plt.grid(True)
    plt.xlabel('Number of clusters')
    plt.ylabel('BSS & WSS')
    plt.xticks(k_values)
    plt.legend()
    plt.title(title);

    plt.savefig(save_path)

### Get results

In [ ]:
#distance_matrix = np.load(f"{results_path}distance_matrix_{metric}{vars}{if_regions}.npy")/vars

In [ ]:
from scipy.cluster.hierarchy import cut_tree

def get_HC_results(metric="HEOM", linkages=["single"], save_linkages=True, save_KE=True, save_dendrogram=False, show_fig=False):
  # if save_linkages:
  #   # may cause RAM issues
  #   distance_matrix = np.load(f"{results_path}distance_matrix_{metric}{vars}{if_regions}.npy")/vars
    
  #   for link in linkages:
  #     print("Saving linkage: ", link, "...")
  #     HC_path = HC_base_path + metric + str(vars) + '/' + algo + sep + metric + sep + link + file_type

  #     Z[metric] = linkage(distance_matrix, link, metric="precomputed")
  #     np.save(HC_path, Z[metric], allow_pickle=False)

  for link in linkages:
    HC_path = f"{HC_base_path}{metric}{vars}{if_regions}/{algo}_{metric}_{link}{file_type}"
    Z = {}
    Z[metric] = np.load(HC_path)

    cluster_labels = fcluster(Z[metric], 5, criterion='maxclust')
    print(np.unique(cluster_labels, return_counts=True))

    cluster_labels = cut_tree(Z[metric], n_clusters=[3])
    print(np.unique(cluster_labels, return_counts=True))

    if save_KE:
      print("Saving Knee-Elbow: ", link, "...")
      wss = []
      bss = []

      
      wss,bss = ComputeWSSBSS(df_DNA, distance_matrix_pair_squared, Z[metric], metric, range(1,20))

      bss_wss_file_type = ".npy"
      BSS_path = f"{HC_base_path}{metric}{vars}{if_regions}/BSS/BSS_{metric}_{link}{bss_wss_file_type}"
      WSS_path = f"{HC_base_path}{metric}{vars}{if_regions}/WSS/WSS_{metric}_{link}{bss_wss_file_type}"
      np.save(WSS_path, wss)
      np.save(BSS_path, bss)

      save_path = f"{pictures_path}{algo}/Knee_Elbow/vars{vars}{if_regions}/"
      ke_file_type = ".png"
      save_path += f"KE_{metric}_{link}{ke_file_type}"
      PlotKneeElbow(bss,wss,range(1,20), metric +' Distance'+' (' + link + ')', save_path)

    if save_dendrogram:
      print("Saving Dendrogram: ", link, "...")
      save_path = f"{pictures_path}{algo}/Dendrogram/"
      dendrogram_file_type = ".html"
      save_path += f"{metric}{vars}{if_regions}/Dendrogram_{link}{dendrogram_file_type}"
      plotDendrogramPlotly(Z[metric], None, save_path=None, show_fig=show_fig, p=3, truncate_mode='lastp', show_heatmap=False)

In [ ]:
get_HC_results(metric, linkages, save_linkages=False, save_KE=False, save_dendrogram=True, show_fig=True)

(array([1], dtype=int32), array([26605]))
(array([0, 1, 2]), array([26540,    27,    38]))
Saving Dendrogram:  single ...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:358: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:359: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:360: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:361: DeprecationWarning:

scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead



(array([1, 2, 3, 4, 5], dtype=int32), array([11674,  7541,  2814,  1208,  3368]))
(array([0, 1, 2]), array([11674, 10355,  4576]))
Saving Dendrogram:  complete ...


(array([1, 2, 3, 4, 5], dtype=int32), array([13306,   382,   228,  1335, 11354]))
(array([0, 1, 2]), array([13306, 12917,   382]))
Saving Dendrogram:  average ...


(array([1, 2, 3, 4, 5], dtype=int32), array([  209, 12855,   729,  4192,  8620]))
(array([0, 1, 2]), array([12812, 13064,   729]))
Saving Dendrogram:  weighted ...


## Plot knee-elbow

In [ ]:
import pandas as pd
import numpy as np

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, inconsistent
from scipy.spatial.distance import cdist, pdist

def vdm_prepare_df(df, cat_ix):
  replace_arr = {}
  for col in df.columns[~cat_ix]:
    replace_arr[col] = { value: key for key, value in enumerate(np.sort(df[col].unique()))}
  return df.replace(replace_arr)

# ------- config vars ---------
algo = "HC"
metric="HVDM"
file_type = ".npy"
vars = 6
vars_list = [7]
range_min = 2
range_max = 30
postprocessing = True
is_postprocessing = "_fix_v2" if postprocessing else ""

base_path = "/content/drive/MyDrive/MIDA2/IntelligentMobilityProject/"
pictures_path = f"{base_path}Pictures/"
dataset_path = f"{base_path}Survey+dataset/"
results_path = f"{base_path}Code/Data/"
HC_base_path = f"{results_path}{algo}/"

# ------- compute wss/bss --------
linkages = ["single", "complete", "average", "weighted"]#, "ward", "centroid", "median"]
for vars in vars_list:
  print(f"vars: {vars}")
  regions = True
  if vars != 7:
    regions = False
  if_regions = '_regions' if regions else ''

  for link in linkages:
    
    bss_wss_file_type = ".npy"
    BSS_path = f"{HC_base_path}{metric}{vars}{if_regions}/BSS/BSS_{metric}_{link}_range({range_min},{range_max}){is_postprocessing}{bss_wss_file_type}"
    WSS_path = f"{HC_base_path}{metric}{vars}{if_regions}/WSS/WSS_{metric}_{link}_range({range_min},{range_max}){is_postprocessing}{bss_wss_file_type}"
    wss = np.load(WSS_path)
    bss = np.load(BSS_path) 

    # Warning: Run this part in a normal cell of colab
    save_path = f"{pictures_path}{algo}/Knee_Elbow/{metric}{vars}{if_regions}/"
    ke_file_type = ".png"
    save_path += f"KE_{metric}_{link}_range({range_min},{range_max}){is_postprocessing}{ke_file_type}"
    PlotKneeElbow(bss,wss,range(range_min, range_max), metric +' Distance'+' (' + link + ')', save_path)
    print(link, "done")
